In [7]:
import os
import numpy as np
import pandas as pd
import pretty_midi
from music21 import converter, chord
from tqdm import tqdm
import warnings

# Data Collection | Pre-Processing | Feature Extraction

In [8]:
# Path to the dataset folder
MIDI_FOLDER = "./dataset" 
CSV_FOLDER = "./features_csv"

# Ensure the CSV folder exists
os.makedirs(CSV_FOLDER, exist_ok=True)

def extract_features(midi_path):
    try:
        midi_data = pretty_midi.PrettyMIDI(midi_path)
        features = {}

        # Extract global features like tempo and number of instruments
        features['tempo'] = np.mean(midi_data.get_tempo_changes()[1])
        features['num_instruments'] = len(midi_data.instruments)

        # Extract note-level features for each instrument
        notes = []
        for instrument in midi_data.instruments:
            for note in instrument.notes:
                notes.append((note.pitch, note.start, note.end, note.velocity))

        # Convert notes to a DataFrame to facilitate further analysis
        if notes:
            df_notes = pd.DataFrame(notes, columns=['Pitch', 'Start', 'End', 'Velocity'])
            features['avg_pitch'] = df_notes['Pitch'].mean()
            features['std_pitch'] = df_notes['Pitch'].std()
            features['avg_velocity'] = df_notes['Velocity'].mean()
            features['std_velocity'] = df_notes['Velocity'].std()

        # Analyze chords using music21
        score = converter.parse(midi_path)
        chords = [ch for ch in score.chordify().recurse() if isinstance(ch, chord.Chord)]
        pitches = [p for ch in chords for p in ch.pitches]
        features['num_chords'] = len(chords)
        features['unique_pitches'] = len(set(pitches))

        return features
    except Exception as e:
        print(f"Error processing {midi_path}: {e}")
        return {}

def process_files():
    csv_path = os.path.join(CSV_FOLDER, 'features.csv')
    composers = ['Bach', 'Beethoven', 'Chopin', 'Mozart']
    all_features = []

    for composer in composers:
        composer_path = os.path.join(MIDI_FOLDER, composer)
        midi_files = [f for f in os.listdir(composer_path) if f.endswith('.mid')]

        for filename in tqdm(midi_files, desc=f"Processing {composer} files"):
            midi_path = os.path.join(composer_path, filename)
            features = extract_features(midi_path)
            if features:
                features['filename'] = filename
                features['composer'] = composer
                all_features.append(features)

    # Save all features to a CSV file
    df = pd.DataFrame(all_features)
    df.to_csv(csv_path, index=False)
    print("Features saved to:", csv_path)

if __name__ == '__main__':
    process_files()

Processing Bach files:  57%|█████▋    | 495/876 [02:56<06:18,  1.01it/s]

# Model Building

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(network_input.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Model Training

In [ ]:
from keras.callbacks import ModelCheckpoint

# Define the checkpoint
checkpoint = ModelCheckpoint("best_model.hdf5", monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model.fit(network_input, network_output, epochs=50, batch_size=64, callbacks=callbacks_list)

# Model Evaluation

In [ ]:
from sklearn.model_selection import train_test_split
from keras.models import load_model
from sklearn.metrics import classification_report, accuracy_score

# Split data into training and test sets
input_train, input_test, output_train, output_test = train_test_split(network_input, network_output, test_size=0.2, random_state=42)

# Train the model
model.fit(input_train, output_train, epochs=50, batch_size=64, validation_data=(input_test, output_test), callbacks=callbacks_list)

# Load the best saved model
best_model = load_model('best_model.hdf5')

# Predictions
predictions = best_model.predict(input_test, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(output_test, axis=1)

# Evaluation metrics
print("Accuracy: ", accuracy_score(true_classes, predicted_classes))
print(classification_report(true_classes, predicted_classes, target_names=label_encoder.classes_))

# Model Optimization

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def create_model():
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(network_input.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(128, return_sequences=True))
    model.add(LSTM(128))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Wrap Keras model so it can be used by scikit-learn
model = KerasClassifier(build_fn=create_model, verbose=1)

# Define the grid search parameters
batch_size = [32, 64, 128]
epochs = [30, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(network_input, network_output)

# Summary
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Confusion Matrix
cm = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix')
plt.ylabel('Actual Classes')
plt.xlabel('Predicted Classes')
plt.show()